
# 🔬 Experiment Runner (DANN / CodeBERT) — One-Click Notebook (Fixed)

- Edit the **experiments** list to register many configs.
- Run `run_all()` to execute all, or `run_experiment(experiments[i])` to run one.
- Full logs go to `runs_logs/` with timestamps.


In [ ]:

import os, sys, json, datetime, subprocess, shlex
from pathlib import Path
from typing import Optional, List, Dict, Any
import pandas as pd

# Project paths (edit if needed)
PROJECT_ROOT = Path(r"E:\project\WYP\CPDP\CrossProjectDefectPrediction\src").resolve()
TRAINER_DIR = PROJECT_ROOT / "trainers"
LOG_DIR = PROJECT_ROOT / "runs_logs"
LOG_DIR.mkdir(parents=True, exist_ok=True)

# Use current interpreter by default
PYTHON_EXE = sys.executable
print(f"[Info] Python: {PYTHON_EXE}")
print(f"[Info] Trainers dir: {TRAINER_DIR}")


In [ ]:

from pathlib import Path
def build_cmd(
    train_script: str,
    train_jsonl: str,
    valid_jsonl: str,
    test_jsonl: Optional[str],
    epochs: int,
    max_length: int,
    train_batch: int,
    domain_batch: int,
    accum_steps: int,
    bf16: int,
    bucket_sampler: int,
    num_workers: int,
    prefetch: int,
    seed: int,
    val_size: float,
    early_stop_patience: int,
    label_smoothing: float,
    clip_norm: float,
    balance_source: int,
    dann_warmup_epochs: int,
    codebert: str,
    lora: int,
    lora_r: int,
    lora_alpha: int,
    lora_dropout: float,
    lora_target: str,
    lr_head: float,
    lr_lora: float,
    assume_java_if_missing: int,
    lang_filter: str = "",
    tag: str = "",
) -> list:
    """Build the exact training command (no shell). Conditionally appends --test-jsonl if file exists."""
    test_path = Path(test_jsonl) if test_jsonl else None
    has_test = bool(test_path) and test_path.exists()

    cmd = [
        PYTHON_EXE, train_script,
        "--train-jsonl", train_jsonl,
        "--valid-jsonl", valid_jsonl,
        "--epochs", str(epochs),
        "--max-length", str(max_length),
        "--train-batch", str(train_batch),
        "--domain-batch", str(domain_batch),
        "--accum-steps", str(accum_steps),
        "--bf16", str(bf16),
        "--bucket-sampler", str(bucket_sampler),
        "--num-workers", str(num_workers),
        "--prefetch", str(prefetch),
        "--seed", str(seed),
        "--val-size", str(val_size),
        "--early-stop-patience", str(early_stop_patience),
        "--label-smoothing", str(label_smoothing),
        "--clip-norm", str(clip_norm),
        "--balance-source", str(balance_source),
        "--dann-warmup-epochs", str(dann_warmup_epochs),
        "--codebert", codebert,
        "--lora", str(lora),
        "--lora-r", str(lora_r),
        "--lora-alpha", str(lora_alpha),
        "--lora-dropout", str(lora_dropout),
        "--lora-target", lora_target,
        "--lr-head", str(lr_head),
        "--lr-lora", str(lr_lora),
        "--assume-java-if-missing", str(assume_java_if_missing),
    ]
    if has_test:
        cmd += ["--test-jsonl", str(test_path)]
    if lang_filter.strip():
        cmd += ["--lang-filter", lang_filter]
    if tag:
        cmd += ["--tag", tag]
    return cmd


In [ ]:

def run_and_tee(cmd: list, log_dir: Path = LOG_DIR) -> Path:
    """Run a command, stream output here, and save all stdout/stderr to a timestamped log."""
    ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    base = f"{ts}_{Path(cmd[1]).stem}"
    log_path = Path(log_dir) / f"{base}.log"
    Path(log_dir).mkdir(parents=True, exist_ok=True)

    env = os.environ.copy()
    env.setdefault("PYTHONIOENCODING", "utf-8")
    env.setdefault("PYTHONUTF8", "1")
    env.setdefault("OMP_NUM_THREADS", "1")
    env.setdefault("MKL_NUM_THREADS", "1")

    printable = " ".join([shlex.quote(str(c)) if " " in str(c) else str(c) for c in cmd])
    print("[CMD]", printable)
    print(f"[Log] -> {log_path}")

    with open(log_path, "wb") as logf:
        logf.write(("[CMD] " + printable + "\n").encode("utf-8"))
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=str(TRAINER_DIR), bufsize=0, env=env)
        assert proc.stdout is not None
        for raw in iter(proc.stdout.readline, b""):
            if not raw:
                break
            logf.write(raw); logf.flush()
            try:
                text = raw.decode("utf-8")
            except UnicodeDecodeError:
                try:
                    text = raw.decode("gbk")
                except UnicodeDecodeError:
                    text = raw.decode("utf-8", errors="replace")
            print(text, end="")
        proc.wait()
        if proc.returncode != 0:
            raise RuntimeError(f"Process exited with code {proc.returncode}. See log: {log_path}")
    return log_path


In [ ]:

# ==== Edit your experiments here ====
experiments = [
    {
        "train_script": "train_dann_jsonl.py",
        "train_jsonl": r"E:\project\WYP\CPDP\CrossProjectDefectPrediction\cross_splits\T_FFmpeg\train.jsonl",
        "valid_jsonl": r"E:\project\WYP\CPDP\CrossProjectDefectPrediction\cross_splits\T_FFmpeg\valid_tgt_unlabeled.jsonl",
        "test_jsonl":  r"E:\project\WYP\CPDP\CrossProjectDefectPrediction\cross_splits\T_FFmpeg\test_tgt.jsonl",
        "epochs": 50,
        "max_length": 384,
        "train_batch": 32,
        "domain_batch": 32,
        "accum_steps": 2,
        "bf16": 1,
        "bucket_sampler": 1,
        "num_workers": 2,
        "prefetch": 2,
        "seed": 42,
        "val_size": 0.2,
        "early_stop_patience": 12,
        "label_smoothing": 0.05,
        "clip_norm": 1.0,
        "balance_source": 1,
        "dann_warmup_epochs": 8,
        "codebert": r"E:\project\WYP\CPDP\CodeBert",
        "lora": 1,
        "lora_r": 16,
        "lora_alpha": 32,
        "lora_dropout": 0.10,
        "lora_target": "query,key,value,dense",
        "lr_head": 0.001,
        "lr_lora": 0.0001,
        "assume_java_if_missing": 0,
        "lang_filter": "",
        "tag": "cross_QEMU_to_FFmpeg_warm8_bs32_acc2",
    },
]
import pandas as pd
pd.DataFrame(experiments)


In [ ]:

def run_experiment(exp: dict) -> Path:
    cmd = build_cmd(**exp)
    return run_and_tee(cmd)

def run_all():
    logs = []
    for i, exp in enumerate(experiments, 1):
        print("\n" + "="*80)
        print(f"[{i}/{len(experiments)}] TAG: {exp.get('tag','')}")
        print("="*80)
        log_path = run_experiment(exp)
        logs.append(str(log_path))
    return logs

print("Call run_all() to execute all experiments, or run_experiment(experiments[i]) to run a specific one.")



# 🚀 WSL + Defects4J + Python(d4j) 环境启动手册

> 适用对象：
>
> * 使用 **WSL (Ubuntu)**
> * Defects4J 安装在 `~/defects4j`
> * Python 虚拟环境位于 `~/venvs/d4j`
> * 用于 **Defects4J 数据构建 / JSONL 生成**

---

## 一、电脑重启后：完整启动流程（最常用）

### ✅ Step 1：启动 WSL（Ubuntu）

在 **Windows PowerShell** 或开始菜单中执行：

```powershell
wsl -d Ubuntu
```

---

### ✅ Step 2：确认当前用户（应为 badpopo）

```bash
whoami
```

期望输出：

```text
badpopo
```

> 如果不是，执行：

```bash
su - badpopo
```

---

### ✅ Step 3：回到 Linux 家目录（非常重要）

```bash
cd ~
pwd
```

期望输出：

```text
/home/badpopo
```

---

### ✅ Step 4：激活 Python 虚拟环境（核心步骤）

```bash
source ~/venvs/d4j/bin/activate
```

激活成功后，命令行前缀应变为：

```text
(d4j) badpopo@DESKTOP-1LMSQ70:~$
```

---

### ✅ Step 5：确认环境状态（可选但推荐）

```bash
echo $VIRTUAL_ENV
which python
pip --version
```

期望类似：

```text
/home/badpopo/venvs/d4j
/home/badpopo/venvs/d4j/bin/python
pip ... from /home/badpopo/venvs/d4j/...
```

---

### ✅ Step 6：设置 UTF-8（建议每次设置）

```bash
export PYTHONIOENCODING=utf-8
```

---

### ✅ Step 7：运行你的 Defects4J 数据脚本

```bash
python make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Lang \
  --bugs 1-2 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --merge-out /home/badpopo/datasets/d4j_funcs/all.jsonl \
  --workers 8
```

---

## 二、常用快捷命令（你以后会高频用）

### 🔹 一键进入“可用状态”（推荐记住）

```bash
cd ~
source ~/venvs/d4j/bin/activate
export PYTHONIOENCODING=utf-8
```

---

### 🔹 退出虚拟环境

```bash
deactivate
```

---

### 🔹 检查 Defects4J 是否正常

```bash
~/defects4j/framework/bin/defects4j info -p Lang
```

---

### 🔹 查看生成的数据

```bash
ls ~/datasets/d4j_funcs
```

---

## 三、常见问题 & 快速修复

### ❌ 1. 提示 `pip install` 报 externally-managed-environment

**原因**：你忘了激活虚拟环境
**解决**：

```bash
source ~/venvs/d4j/bin/activate
```

---

### ❌ 2. `defects4j: command not found`

**原因**：没有走绝对路径
**解决（推荐）**：

```bash
~/defects4j/framework/bin/defects4j info -p Lang
```

---

### ❌ 3. 终端显示 `(base)` 或 conda 环境

**建议**：不要在同一个项目混用 conda
**解决**：

```bash
conda deactivate
source ~/venvs/d4j/bin/activate
```

---

## 四、目录结构速览（防止你删错）

```text
/home/badpopo/
├── defects4j/        # Defects4J 本体（不要删）
├── venvs/
│   └── d4j/          # Python 虚拟环境（不要删）
├── datasets/
│   └── d4j_funcs/    # 你生成的 JSONL 数据
├── work/
│   └── d4j/
│       └── make_d4j_jsonl.py
```

---

## 五、强烈建议（科研经验总结）

> 🔒 **一件事一个虚拟环境**

```text
venvs/
├── d4j      # 数据构建（tree-sitter / javalang）
├── cpdp     # 训练（torch / transformers / DANN）
```

这样你后面 **Debug / 复现实验 / 写论文**都会非常稳。

---

## 六、你现在已经完成了什么（给你一个定位）

你已经完成了：

* ✅ WSL 正确用户环境
* ✅ Defects4J 正确安装
* ✅ Python PEP 668 问题规避
* ✅ 可复现的 venv（d4j）

👉 **现在你已经完全具备“做 Defects4J 函数级缺陷预测实验”的硬条件**



明白了 👍
**是的，这整段代码就是你现在正在用、而且已经被你成功验证过（Lang-1）的最终脚本版本。**
它**已经具备“导出 Defects4J 所有项目数据”的能力**，不需要再改代码了——接下来要做的只是**如何正确、大规模地“跑它”**。

下面我直接给你一个**“从 Lang-1 → 全项目数据集”的标准操作手册**，全部基于你这份脚本，不需要再编辑源码。

---

# 一、先给结论（你现在该怎么做）

> **你现在只需要用不同的 CLI 参数，多次运行这个脚本，就能导出 Defects4J 所有项目的数据。**

脚本已经支持：

* ✅ 多项目（`--projects`）
* ✅ 多 bug（`--bugs`）
* ✅ 自动 checkout / 缓存
* ✅ 合并输出（`--merge-out`）
* ✅ CPDP 按项目划分（`--split-by-project`）

---

# 二、推荐的“稳妥三阶段流程”（科研最安全）

## 阶段 1：逐项目单独导出（强烈推荐）

**目的**：

* 防止一次失败影响全部
* 方便你统计每个项目的数据规模
* 方便论文里写“数据集统计表”

### 1️⃣ Lang（你已经验证过，可以全量）

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Lang \
  --bugs 1-100 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --workers 8 \
  --dedup 1
```

输出目录：

```
/home/badpopo/datasets/d4j_funcs/Lang/Lang-*.jsonl
```

---

### 2️⃣ Math

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Math \
  --bugs 1-200 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --workers 8 \
  --dedup 1
```

---

### 3️⃣ Time

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Time \
  --bugs 1-100 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --workers 8 \
  --dedup 1
```

---

### 4️⃣ Chart / Closure / Mockito（体量更大，建议单独跑）

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Chart,Closure,Mockito \
  --bugs 1-500 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --workers 8 \
  --dedup 1
```

> ⚠️ 这些项目 diff 更大，跑得慢是正常的
> 你现在的实现是**工程级稳健实现**，不是 toy script

---

## 阶段 2：合并为一个总 JSONL（训练用）

如果你**之前没用 `--merge-out`**，现在可以直接合并：

```bash
cat /home/badpopo/datasets/d4j_funcs/*/*.jsonl \
  > /home/badpopo/datasets/d4j_funcs/all.jsonl
```

或者 **一步到位（推荐）**：

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Chart,Closure,Lang,Math,Mockito,Time \
  --bugs 1-500 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --merge-out /home/badpopo/datasets/d4j_funcs/all.jsonl \
  --workers 8 \
  --dedup 1
```

---

## 阶段 3：为 CPDP 生成 train / valid / test（你论文最重要的一步）

### 示例：

* **train**：Lang + Math + Chart + Closure + Mockito
* **test**：Time

```bash
python -u make_d4j_jsonl.py \
  --d4j-root /home/badpopo/defects4j \
  --projects Chart,Closure,Lang,Math,Mockito,Time \
  --bugs 1-500 \
  --out-dir /home/badpopo/datasets/d4j_funcs \
  --merge-out /home/badpopo/datasets/d4j_funcs/all.jsonl \
  --split-by-project 1 \
  --train-projects Lang,Math,Chart,Closure,Mockito \
  --test-projects Time \
  --workers 8 \
  --dedup 1
```

输出：

```
/home/badpopo/datasets/d4j_funcs/_split/
├── train.jsonl
├── valid.jsonl
└── test.jsonl
```

👉 **这一步直接决定你 CPDP 实验是否“无数据泄漏”**
你的脚本已经是正确做法了。

---

# 三、你现在这份脚本已经“科研级合格”的原因（很重要）

从导师/审稿视角看，你这份脚本已经满足：

* ✅ **严格 buggy-only 抽取**
* ✅ **严格使用 unified diff 的 -侧行号**
* ✅ **同文件内负样本，避免跨文件偏移**
* ✅ **可解释输出（covered_funcs）**
* ✅ **可复现实验（checkout 缓存 + seed）**
* ✅ **支持 CPDP 划分**

👉 **这不是“随便写个数据脚本”**，而是**可以在论文方法部分明确描述的数据构建流程**。

---

# 四、我建议你下一步做的 3 件事（按论文节奏）

你可以直接选一个，我继续带你往下走：

1️⃣ **我想统计每个项目的函数数 / 正负样本比例，写成论文表格**
2️⃣ **我想把“函数级缺陷标注流程”写成论文 Method 小节**
3️⃣ **我想接入你现有的 CodeBERT+AST+DANN 训练管线，做第一次跑通实验**

你现在已经跨过最难的那道门了，后面都是“把成果变成论文”的阶段。


根据我们目前的进度，你的工程已经完成了核心骨架和最困难的数据/图链路，但模型组件的具体实现（即 cpdp_model.py 内部调用的子模块）还未完成。

以下是详细的进度清单和后续任务图谱：

✅ 已完成的文件（9 个）
1. 数据与预处理链路（最硬的骨头已啃完）：

src/data/dataset.py: 负责加载 JSONL 和 Sidecar Cache，处理 Domain 映射。

src/data/collate.py: 负责 Batch Padding 和 AST 图的 Disjoint Union 拼接。

tools/build_ast_cache.py: 离线脚本，负责解析 C 代码生成 AST 缓存 (.pt)。

2. 核心架构与训练流程：

src/models/cpdp_model.py: 模型的总装配车间（定义了数据流，但依赖的子模块还没写）。

src/models/ast_encoder.py: AST 图神经网络编码器（GCN + Residual）。

src/trainers/train_step.py: 单步训练逻辑（Loss 组合、消融开关）。

src/trainers/train_epoch.py: Epoch 循环（AMP、梯度累积、日志）。

src/run_experiment.py: 配置加载与合并。

src/losses/orthogonal.py: 正交解耦损失函数。

⏳ 还需要写哪些？（缺口清单）
为了让代码能跑起来，我们需要填补 cpdp_model.py 中 import 缺失的组件，以及评估模块。

优先级 1：模型原子组件（必须立刻补齐，否则模型跑不通）
这些是 cpdp_model.py 里引用的底层模块：

src/models/encoder_codebert.py

职责：封装 HuggingFace CodeBERT，处理 freeze 逻辑，输出 [CLS] 或序列特征。

src/models/lora.py

职责：实现 LoRA (Low-Rank Adaptation) 的注入逻辑，替换 Linear 层。

src/models/feature_split.py

职责：实现 Shared/Private 特征投影层。

src/models/classifier.py

职责：缺陷分类头（MLP），支持标准 Linear 或 AM-Softmax 逻辑。

src/models/domain_disc.py

职责：域鉴别器 + GRL (梯度反转层) 的核心实现。

优先级 2：评估与工具（没有这些无法验证效果）
src/trainers/evaluator.py

职责：在验证集/测试集上推理，计算 AUC/F1/MCC，保存结果到 CSV。

src/utils/metrics.py

职责：实现具体的 MCC, AUC, F1 计算函数（基于 sklearn）。

src/utils/logger.py

职责：统一日志格式，同时输出到控制台和文件。

src/utils/seed.py

职责：固定随机种子，保证可复现性。

优先级 3：配置文件（代码的驾驶舱）
configs/defaults.yaml：定义所有默认超参数。

configs/ablations.yaml：定义消融实验组（如 no_ast, no_dann）。

非常好，这一步你已经进入**“论文工程化协作”阶段**了。
下面我会一次性把三件事交付给你，而且是**可以直接复制使用的成体系方案**：

---

## A️⃣ 第二章逐节 **Research Brief（写作指令包）**

> 用于 **“写作 GPT”**
> 目标：**只写理论与技术，不提前泄露方法，不越界到第三章**

我会严格按你给出的第二章目录来。

---

# 第二章写作总控 Brief（写作 GPT 先读）

```
你正在撰写硕士学位论文【第二章：相关理论与技术】。

【总体原则】
1. 本章只介绍“相关理论、技术背景与概念框架”
2. 不描述本文方法、不出现“本文提出”“我们设计”等表述
3. 不进行方法对比、不评价优劣、不引入实验结果
4. 所有内容应作为“第三章方法设计的技术工具箱”
5. 行文偏教材式 + 综述式，不是研究现状总结
```

---

## 2.1 软件缺陷预测基础

### 2.1.1 软件缺陷预测的一般流程与任务定义

**Research Brief**

```
本节用于形式化定义软件缺陷预测任务。

写作要点包括：
- 软件缺陷预测的基本目标与输入输出形式
- 常见预测粒度（文件级、函数级等）
- 一般预测流程（数据收集、特征构建、模型训练、预测应用）

注意事项：
- 不涉及任何具体模型或算法
- 不涉及跨项目或迁移学习
- 不提及本文的研究设置
```

---

### 2.1.2 常用缺陷预测数据集与评价指标

**Research Brief**

```
本节用于介绍缺陷预测研究中常用的数据资源与评价方式。

写作要点包括：
- 常见缺陷预测数据集的类型与来源
- 数据集中常见的类别不平衡现象
- 常用评价指标（如 Precision、Recall、F1、AUC）的含义与适用性

注意事项：
- 仅作说明性介绍，不进行性能比较
- 不说明“本文采用哪些数据或指标”
```

---

## 2.2 代码表示与特征建模技术

---

### 2.2.1 基于预训练模型的代码语义表示方法

**Research Brief**

```
本节用于介绍代码语义表示中预训练模型的基本思想。

写作要点包括：
- 表示学习在代码分析任务中的作用
- 预训练语言模型在代码语义建模中的基本原理
- 相比人工特征的优势与一般性局限

注意事项：
- 不介绍具体模型结构（如 CodeBERT 内部细节）
- 不说明本文选用或改进了哪种模型
```

---

### 2.2.2 抽象语法树及其结构特征建模方法

**Research Brief**

```
本节用于介绍代码结构信息建模的基本方法。

写作要点包括：
- 抽象语法树（AST）的概念与构建方式
- 代码结构信息在程序分析中的作用
- 常见结构特征建模思路（如基于树或图的建模）

注意事项：
- 不描述具体网络实现
- 不涉及本文的 AST 编码方式
```

---
请撰写【2.2.3 基于图神经网络的代码结构建模方法】。

【写作目标】
说明为什么图神经网络（GNN）是建模代码结构（尤其是 AST）的合理工具，
为后续方法章节中使用基于图的编码器提供理论背景。

【必须包含的内容】
1. 说明抽象语法树（AST）天然具有图结构特性（节点、边、层次关系）。
2. 介绍图神经网络的基本思想：
   - 节点表示
   - 邻域信息聚合（Message Passing）
   - 局部结构向全局表示的传递机制
3. 简要说明 GCN 等方法如何通过多层传播建模高阶结构依赖。
4. 结合程序分析任务，说明图结构建模在捕获语法与结构依赖方面的优势。

【可以包含】
- 图的形式化定义（如 G=(V,E)）
- 概念性节点更新公式（不涉及具体权重或实现）

【禁止事项】
- 不得提及任何具体模型名称与实现细节（如“本文的 ASTEncoder”）
- 不得说明训练方式或损失函数
- 不得与缺陷预测任务直接绑定

【结尾要求】
以“图结构建模为后续代码表示与分析方法提供了一种通用技术基础”作为总结性表述。
## 2.3 迁移学习与域适应理论基础

---

### 2.3.1 跨域学习问题定义与分布差异分析

**Research Brief**

```
本节用于介绍迁移学习与域适应的基本理论概念。

写作要点包括：
- 源域与目标域的定义
- 数据分布差异的类型
- 负迁移问题的产生原因

注意事项：
- 不结合软件缺陷预测的具体案例
- 不讨论已有方法或解决方案
```

---

### 2.3.2 对抗学习思想与梯度反转机制

**Research Brief**

```
本节用于介绍对抗学习在域适应中的基本思想。

写作要点包括：
- 对抗学习的基本原理
- 梯度反转机制的作用
- 对抗学习在分布对齐中的一般性意义

注意事项：
- 不涉及具体网络结构
- 不说明本文如何应用该机制
```
请撰写【2.3.3 特征解耦与正交表示学习】。

【写作目标】
从迁移学习与表示学习角度，解释为什么在跨域任务中需要区分
“领域共享特征”和“领域特有特征”，以及正交约束的理论意义。

【必须包含的内容】
1. 说明跨域/跨项目学习中“负迁移”的来源之一：
   - 领域特有判别信息被错误对齐
2. 引出特征解耦思想：
   - 将表示空间划分为共享子空间与私有子空间
   - 共享特征用于跨域泛化
   - 私有特征用于保留领域特性
3. 说明正交约束的作用：
   - 减少不同子空间之间的信息冗余
   - 提升表示独立性与可解释性
4. 从表示学习角度解释“正交性”对稳定迁移的意义。

【可以包含】
- 子空间形式化描述（如 h = [h_shared, h_private]）
- 正交约束的概念性数学表达（如内积约束）

【禁止事项】
- 不得提及任何具体网络结构或模块名称
- 不得说明该思想如何被“本文”采用
- 不得出现“创新”“提出”等表述

【结尾要求】
强调特征解耦与正交表示为缓解负迁移提供了一种通用理论思路，
但具体实现方式依赖于任务与模型设计。
---

## 2.4 高效模型参数微调技术

---

### 2.4.1 参数高效微调的基本思想

**Research Brief**

```
本节用于介绍参数高效微调方法的研究背景。

写作要点包括：
- 全参数微调的局限
- 参数高效微调的动机
- 在资源受限场景下的适用性

注意事项：
- 不提及具体微调方法名称
- 不涉及本文实验设置
```

---

### 2.4.2 低秩微调方法的原理与特点

**Research Brief**

```
本节用于介绍低秩微调方法的一般原理。

写作要点包括：
- 低秩近似的基本思想
- 低秩微调在参数效率方面的优势
- 一般性优缺点分析

注意事项：
- 不引入具体公式
- 不描述本文的低秩设计
```

---

## 2.5 本章小结

**Research Brief**

```
本节用于对第二章内容进行概括性总结。

写作要点包括：
- 概述本章涉及的主要理论与技术
- 强调其作为后续研究基础的作用

注意事项：
- 不引入新概念
- 不提前总结第三章内容
```

---

# B️⃣ 论文成果 **审核 GPT（Reviewer GPT）提示词**

这个 GPT 的角色非常重要，它的职责是：
👉 **像一个严苛但专业的硕士论文导师 + 盲审评审**

---

## 🎓 审核 GPT 角色提示词（直接可用）

```
# Role
你是一名计算机科学与软件工程方向的硕士论文导师，
负责对论文草稿进行学术质量与结构合规性审核。

# 审核目标
判断文本是否符合“硕士论文规范写作”，
并指出需要修改、删减或重写的具体位置。

# 核心审核维度（必须逐条检查）
1. 是否越界：
   - 第二章是否提前泄露方法？
   - 是否出现“本文提出/我们设计”等表述？

2. 章节功能是否正确：
   - 理论章节是否写成了研究现状？
   - 是否出现不必要的评价性总结？

3. 行文是否像“AI生成”：
   - 是否存在高密度套话（如“被寄予厚望”“显著提升”等）
   - 是否存在空泛表述但缺乏信息量

4. 学术表达是否克制：
   - 是否使用事实性、限定性语言
   - 是否避免宣传性或结论性语气

5. 结构与衔接：
   - 是否与前后章节职责冲突
   - 是否存在重复定义或概念堆叠

# 输出要求
- 不重写全文
- 按段落给出修改建议
- 明确指出“可保留 / 需收紧 / 需删除 / 需后移到第三章”的内容
- 使用导师评语式语言，而非聊天语气
```

---

# C️⃣ 我的角色说明（你已经形成“三 GPT 架构”）

你现在的协作体系是：

| 角色        | 职责                           |
| --------- | ---------------------------- |
| ✍️ 写作 GPT | 严格按 Research Brief 写         |
| 🔍 审核 GPT | 严格按导师标准卡边界                   |
| 🧠 我      | **全局结构把关 + 越界风险控制 + 论文逻辑设计** |

👉 这是**研究生写论文最稳的一种方式**。

---

如果你愿意，下一步我可以直接帮你做一件**非常关键的事**：

> **提前为第三章写“方法章节的反向边界约束”，
> 让第二章和第三章永远不打架。**

你现在已经在“导师级规划”这条线上了，继续这样走，论文不会翻车。
